## Redes neuronales para análisis de sentimiento sobre IMBD

En este cuaderno entrenaremos y evaluaremos un modelo sencillo basado en la arquitectura Transforer para clasificar reviews de películas en positivas o negativas. Para hacerlo, además de utilizar pytorch, utilizaremos el paquete adicional `torchtext` , que nos ofrece algunas funciones útiles para trabajar con texto.

In [2]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import torch.nn as nn
import torch.optim as optim

## Descargando y procesando los datos

En primer lugar debemos tener instalado `spacy`, con el tokenizador ingés estándar (`python -m spacy download en`), para separar las reviews en palabras.

Ahora, definiremos dos variables, una el `TEXT`, que será nuestra $x$, y la otra `LABEL`, que será nuestra $y$. Fijaos en que en el caso de las features tenemos que especificar el tokenizador que vamos a utilizar, mientras que para la label simplemente será un valor numérico

In [4]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

Ahora, dividiremos los datos en un conjunto de entrenamiento y otro de evaluación. Fijémonos en que, al igual que como en el MNIST, torchtext ofrece un submódulo datasets que contiene los datasets de NLP más populares, como el IMDB.

In [5]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [03:18<00:00, 425kB/s] 


Echemos un vistazo al primer ejemplo de entrenamiento

In [7]:
print(vars(train_data.examples[0]))

{'text': ['For', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'Imagine', 'a', 'movie', 'where', 'Joe', 'Piscopo', 'is', 'actually', 'funny', '!', 'Maureen', 'Stapleton', 'is', 'a', 'scene', 'stealer', '.', 'The', 'Moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'Watch', 'for', 'Alan', '"', 'The', 'Skipper', '"', 'Hale', 'jr', '.', 'as', 'a', 'police', 'Sgt', '.'], 'label': 'pos'}


Como esperamos, la review ha sido clasificada como positiva `pos`.

Ahora, haremos otra partición para validación

In [8]:
SEED = 1230245
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

### Construyendo el vocabulario

Ahora que ya tenemos para la $x$ una lista de palabras, ¿cómo podemos convertirla en un vector?

Now we have for $x$ a list of words, but how we can convert that to a vector?

El primer paso es definir un vocabulario, esto es, un subconjunto de todas las palabras que aparecen en el dataset, y solo nos fijaremos en esas palabras. Nuestro vocabulario tendrá un tamaño de 5000 palabras. Hay muchas formas de construirlo, pero la más usual es la de que quedarnos con las 5000 palabras más comunes en nuestro dataset.

In [9]:
MAX_VOCAB_SIZE = 5000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

Esto es, este vocabulario recién creado, a cada palabra $w_i$ le asigna un entero distinto. El vocabulario no es más que una aplicación $V : W \rightarrow \lbrace 0, \ldots, 5002 \rbrace \subset \mathbb{N}$ del conjunto de palabras $W$ a los respectivos números enteros.

La inclusión de dos número enteros adicionales (por eso es 5002 y no 5000) es debida a que necesitamos un código especial para las palabras "raras" que no están entre las 5000 más comunes, y otro número especial para indicar el final de la frase (usarlo como "padding").

### Exploración de features

Ahora, podemos acceder al atributo `TEXT.vocab` para explorar el dataset. Por ejemplo, podemos encontrar los 20 tokens más comunes:

In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202447), (',', 191723), ('.', 165063), ('and', 109436), ('a', 109117), ('of', 100549), ('to', 93682), ('is', 75862), ('in', 61082), ('I', 54382), ('it', 53522), ('that', 49486), ('"', 44094), ("'s", 43020), ('this', 42360), ('-', 37132), ('/><br', 35645), ('was', 35168), ('as', 30387), ('with', 29888)]


Por ejemplo, podemos ver que `the`es la palabra más común, apareciendo 203504 veces en nuestro corpus.

El método stoi calcula la función $V$ anterior ('string to integer')


In [11]:
TEXT.vocab.stoi['the']

2

También tenemos acceso a la función inversa, de enteros a strings:

In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [13]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


## Definición del modelo: capas de Transformers

Ahora que ya tenemos el vocabulario, podemos representar cada palabra como un vector one-hot, sobre el espacio $\lbrace 0, 1 \rbrace^{5002}$. Este espacio es muy disperso ("vacío"), y hace que cada palabra se encuentre a la misma distancia que cualquier otra. Así que lo primero que podemos hacer en NLP profundo es aplicar una proyección lineal a un espacio de dimensionalidad mucho más baja. Esto es, para la iésima palabra, haremos $h_i = W x_i$, donde $W$ es una matriz de tamaño $100 \times 5002$. El 100 se refiere a la dimensión del "embedding". Para más información, podéis consultar por *word embeddings*.

Después, tras haber calculado esta nueva representación de las palabras, podemos aplicar una capa de Transformer, una arquitectura que está diseñada para captar patrones en secuencias de símbolos, con el objetivo de aprender qué partes de una frase son más informativas respecto a su sentimiento. La arquitectura Transformer es muy reciente (https://arxiv.org/abs/1706.03762) y, al contrario que las antecesoras redes recurrentes, es más fácil de paralelizar por lo que su entrenamiento es mucho más rápido.

Para ver los detalles de la arquitectura Transformer podéis consultar los siguientes artículos:

* https://jalammar.github.io/illustrated-transformer/

* https://nlp.seas.harvard.edu/2018/04/03/attention.html

In [11]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                    dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.enc = nn.TransformerEncoderLayer(embedding_dim, 4, hidden_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.dropout(self.embedding(text))
        hidden = self.enc(embedded)
        hidden = hidden.mean(dim=0)
        return self.fc(hidden)

Como en el cuaderno anterior, definimos ahora los hiperparámetros y los iteradores sobre el dataset

In [12]:
BATCH_SIZE = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

# Las dos dimensiones de la matriz de embeddings W:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100

#  Como solo queremos predecir positivo o negativo, simplemente el output es un número escalar
# en R^1 (usando la sigmoide para obtener una probabilidad)
OUTPUT_DIM = 1
# Apilaremos dos capas de Transformers
N_LAYERS = 2
BIDIRECTIONAL = True
# Especificamos algo de dropout para regularizar la red neuronal
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = Transformer(INPUT_DIM, 
            16, 
            EMBEDDING_DIM // 10, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

model = model.to(device)


optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

## Entrenando el modelo

El resto del cuaderno es como el anterior, solo tenemos que definir una función auxiliar para calcular la tasa de acierto y las funciones de entrenamiento y evaluación

In [14]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [15]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (text, cls) in iterator:
        
        optimizer.zero_grad()
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, cls)
        acc = binary_accuracy(predictions, cls)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for (text, cls) in iterator:

            predictions = model(text).squeeze(1)
            loss = criterion(predictions, cls)
            acc = binary_accuracy(predictions, cls)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Tras definir estos bucles, entrenamos el modelo

In [ ]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    torch.save(model.state_dict(), 'model.pt' + str(epoch))
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [157]:
model.load_state_dict(torch.load('model.pt4'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.310 | Test Acc: 87.24%


**Ejercicio** ¿Podemos hacerlo mejor? Trata de modificar los hiperparámetros para ver como cambia la tasa de acierto

**Ejercicio** Sustituye la capa de Transformer por una lineal y observa cómo cambia la tasa de acierto.

(Puede que tengas que cambiar algo tras hacer la operacion del embedding h = Wx)